In [1]:
! python3 --version

Python 3.7.4


In [3]:
! pip3 install pandas

     |████████████████████████████████| 10.4MB 1.3MB/s eta 0:00:01    |███▎                            | 1.1MB 1.3MB/s eta 0:00:08     |████▊                           | 1.5MB 1.3MB/s eta 0:00:07
     |████████████████████████████████| 512kB 25.4MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import pandas as pd

In [2]:
! ls ../thesis/

ls: cannot access '../thesis/': No such file or directory


In [14]:
metadata_path = "/mnt/ceph/cme/30Ketds/30Kmetadata.json"
#f = open(metadata_path)
#f.read()
md_df = pd.read_json(metadata_path, orient=str, lines=True)
abstracts = md_df[['identifier-uri', 'description-abstract']]
#abstracts.iloc[0]

In [17]:
abstracts

,identifier-uri,description-abstract
0,http://hdl.handle.net/10919/9558,The Federal Communications Commission (FCC) ha...
1,http://hdl.handle.net/10919/9559,"For humans, walking is the principle means of ..."
2,http://hdl.handle.net/10919/9560,This thesis is an attempt to reconcile the for...
3,http://hdl.handle.net/10919/9561,The light is the giver of presence. Nature is ...
4,http://hdl.handle.net/10919/9562,"Since Senegal's independence in 1960, groundnu..."
...,...,...
30956,http://hdl.handle.net/10919/83363,As the number of public school students identi...
30957,http://hdl.handle.net/10919/83364,This dissertation examines the emerging practi...
30958,http://hdl.handle.net/10919/83372,Switch mode power supplies are widely used in ...
30959,http://hdl.handle.net/10919/83390,As a result of recent federal legislative chan...


In [3]:
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

import os
import string

[nltk_data] Downloading package punkt to /home/pkmandke/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pkmandke/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
N = 10000
#ETD_PATH = '/mnt/ceph/cme/thesis/'
ETD_PATH = '../../thesis/'
list_of_docs = os.listdir(ETD_PATH)[:2]
len(list_of_docs)

2

In [43]:
def data_generator():
    
    for idx, folder in enumerate(list_of_docs):
        for _ in os.listdir(ETD_PATH + folder):
            file_name = ''
            if _.endswith('.pdf.txt') or _.endswith('.PDF.txt'):
                file_name = _
                break
        if file_name:
            with open(ETD_PATH + folder + '/' + file_name) as f:
                document = f.read()
                
                tokens = nltk.word_tokenize(document.lower().translate(str.maketrans('', '', string.punctuation)))
                stems = []
                
                for item in tokens:
                    if item in stopwords.words():
                        continue
                    stems.append(PorterStemmer().stem(item))

                #tokens = gensim.utils.simple_preprocess(document)
                
                yield gensim.models.doc2vec.TaggedDocument(stems, [folder])

In [44]:
train_corpus = list(data_generator())
len(train_corpus), type(train_corpus[0])

(2, gensim.models.doc2vec.TaggedDocument)

In [49]:
train_corpus[1][1]

['10001']

In [46]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=20, min_count=2, epochs=5, workers=5)
model.build_vocab(train_corpus)

In [47]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [48]:
model.docvecs[1]

array([  2.0238445,  -3.645342 , -10.700458 ,  10.126007 ,  -7.6178694,
        -9.052647 ,  -7.9369607,   0.8680063,  -0.5970208,   5.070162 ,
       -15.632132 ,   1.2702698,  -1.9495729,  -2.3695042,  -9.203182 ,
        -7.465273 ,   6.878441 ,  11.649819 ,  10.651127 ,   4.1084332],
      dtype=float32)

In [59]:
ranks = []
second_ranks = []
for doc_id, name in zip(range(2), ['10000', '10001']):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(name)
    ranks.append(rank)

    second_ranks.append(sims[0])

In [60]:
second_ranks

[('10000', 0.9977684020996094), ('10001', 0.9994410872459412)]

In [41]:
model.estimate_memory()

{'doctag_lookup': 400,
 'doctag_syn0': 160,
 'vocab': 1074500,
 'vectors': 171920,
 'syn1neg': 171920,
 'total': 1418900}

In [ ]:
model.save('../obj/doc2vec1')